In [1]:
import pandas as pd 
import os
from Functions import evaluate

In [2]:
os.listdir("./data/evaluate/raw/")

['.ipynb_checkpoints',
 'cast.csv',
 'clean_movies.csv',
 'crew.csv',
 'genres.csv',
 'keywords.csv',
 'prod_company.csv']

In [3]:
movies = pd.read_csv('./data/evaluate/raw/clean_movies.csv')

In [4]:
movies.head(2)

,id,adult,original_title,runtime,genres,popularity,production_companies,release_date,revenue,vote_average
0,862,False,Toy Story,81.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30 00:00:00.000000000,373554033.0,7.7
1,8844,False,Jumanji,104.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15 00:00:00.000000000,262797249.0,6.9


In [5]:
movies.columns

Index(['id', 'adult', 'original_title', 'runtime', 'genres', 'popularity',
       'production_companies', 'release_date', 'revenue', 'vote_average'],
      dtype='object')

In [6]:
cols_needed = ['id', 'adult', 'original_title', 'runtime',
               'popularity','revenue', 'vote_average','release_date']
movies = movies[cols_needed]

In [7]:
movies.head(2)

,id,adult,original_title,runtime,popularity,revenue,vote_average,release_date
0,862,False,Toy Story,81.0,21.946943,373554033.0,7.7,1995-10-30 00:00:00.000000000
1,8844,False,Jumanji,104.0,17.015539,262797249.0,6.9,1995-12-15 00:00:00.000000000


In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45446 entries, 0 to 45445
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              45446 non-null  int64  
 1   adult           45446 non-null  bool   
 2   original_title  45446 non-null  object 
 3   runtime         45446 non-null  float64
 4   popularity      45446 non-null  float64
 5   revenue         45446 non-null  float64
 6   vote_average    45446 non-null  float64
 7   release_date    45446 non-null  object 
dtypes: bool(1), float64(4), int64(1), object(2)
memory usage: 2.5+ MB


In [9]:
movies.rename(columns={"id" : 'movie_id'}, inplace=True)

In [10]:
movies['release_date'] = pd.to_datetime(movies.release_date)

## Add Dim ID

In [11]:
os.listdir("./data/out/")

['.ipynb_checkpoints',
 'Dim-Cast.csv',
 'Dim-Crew.csv',
 'Dim-Dates.csv',
 'Dim-genres.csv',
 'Dim-keywords.csv',
 'Dim-Production-company.csv',
 'Fact.csv']

In [12]:
cast = pd.read_csv('./data/out/Dim-Cast.csv')
cast.columns

Index(['character', 'name', 'order', 'movie_id', 'DimCastID'], dtype='object')

In [13]:
movies = movies.merge(cast[['movie_id','DimCastID']] , how='left', on = 'movie_id').drop_duplicates()

In [14]:
crew = pd.read_csv('./data/out/Dim-Crew.csv')
crew.columns

Index(['name', 'department', 'job', 'movie_id', 'DimCrewID'], dtype='object')

In [15]:
movies = movies.merge(crew[['movie_id','DimCrewID']] , how='left', on = 'movie_id').drop_duplicates()

In [16]:
dates = pd.read_csv('./data/out/Dim-Dates.csv')
dates.rename(columns={"id" : "movie_id"},inplace=True)
dates.columns

Index(['movie_id', 'release_date', 'year', 'quarter', 'month', 'day',
       'DimDateID'],
      dtype='object')

In [17]:
movies = movies.merge(dates[['movie_id','DimDateID']] , how='left', on = 'movie_id').drop_duplicates()

In [18]:
genre = pd.read_csv('./data/out/Dim-genres.csv')
genre.columns

Index(['name', 'movie_id', 'DimGenreID'], dtype='object')

In [19]:
movies = movies.merge(genre[['movie_id','DimGenreID']] , how='left', on = 'movie_id').drop_duplicates()

In [20]:
keywords = pd.read_csv('./data/out/Dim-keywords.csv')
keywords.columns

Index(['name', 'movie_id', 'DimKeywordsID'], dtype='object')

In [21]:
movies = movies.merge(keywords[['movie_id','DimKeywordsID']] , how='left', on = 'movie_id').drop_duplicates()

In [22]:
pc = pd.read_csv('./data/out/Dim-Production-company.csv')
pc.columns

Index(['name', 'movie_id', 'DimProdCompID'], dtype='object')

In [23]:
movies = movies.merge(pc[['movie_id','DimProdCompID']] , how='left', on = 'movie_id').drop_duplicates()

In [24]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45446 entries, 0 to 82372
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   movie_id        45446 non-null  int64         
 1   adult           45446 non-null  bool          
 2   original_title  45446 non-null  object        
 3   runtime         45446 non-null  float64       
 4   popularity      45446 non-null  float64       
 5   revenue         45446 non-null  float64       
 6   vote_average    45446 non-null  float64       
 7   release_date    45446 non-null  datetime64[ns]
 8   DimCastID       43031 non-null  float64       
 9   DimCrewID       44674 non-null  float64       
 10  DimDateID       45446 non-null  int64         
 11  DimGenreID      43004 non-null  float64       
 12  DimKeywordsID   31103 non-null  float64       
 13  DimProdCompID   33572 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(9), int64(2), o

In [25]:
# Replace nan with -1 
# the Dim tables Contain ID -1 With Not Provided Data 

In [26]:
movies.fillna(-1, inplace=True)

In [27]:
movies

,movie_id,adult,original_title,runtime,popularity,revenue,vote_average,release_date,DimCastID,DimCrewID,DimDateID,DimGenreID,DimKeywordsID,DimProdCompID
0,862,False,Toy Story,81.0,21.946943,373554033.0,7.7,1995-10-30 00:00:00.000000000,16223.0,263.0,1,6801.0,4872.0,22856.0
1,8844,False,Jumanji,104.0,17.015539,262797249.0,6.9,1995-12-15 00:00:00.000000000,3674.0,7013.0,2,5172.0,9189.0,5435.0
4,15602,False,Grumpier Old Men,101.0,11.712900,0.0,6.5,1995-12-22 00:00:00.000000000,29060.0,25692.0,3,19431.0,15085.0,15661.0
6,31357,False,Waiting to Exhale,127.0,3.859495,81452156.0,6.1,1995-12-22 00:00:00.000000000,22681.0,14045.0,4,13942.0,10481.0,26432.0
7,11862,False,Father of the Bride Part II,106.0,8.387519,76578911.0,5.7,1995-02-10 00:00:00.000000000,18414.0,20961.0,5,29138.0,5180.0,15659.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82368,439050,False,رگ خواب,90.0,0.072051,0.0,4.0,2262-04-11 23:47:16.854775807,40268.0,16972.0,45429,19430.0,25125.0,-1.0
82369,111109,False,Siglo ng Pagluluwal,360.0,0.178241,0.0,9.0,2011-11-17 00:00:00.000000000,19324.0,21491.0,45430,29180.0,17700.0,17981.0
82370,67758,False,Betrayal,90.0,0.903007,0.0,3.8,2003-08-01 00:00:00.000000000,13072.0,25035.0,45431,10921.0,-1.0,17980.0
82371,227506,False,Satana likuyushchiy,87.0,0.003503,0.0,0.0,1917-10-21 00:00:00.000000000,36759.0,35285.0,45432,-1.0,-1.0,33562.0


### Add Normalized Revenue Against a Customer Price Index (CPI)

In [28]:
cpi = pd.read_csv('data/in/fred/Consumer Price Index of Admission to Movies.csv')

In [29]:
cpi['data'] = pd.to_datetime(cpi.date)

In [30]:
cpi.date.min(), cpi.date.max()

('1999-01-01', '2022-08-01')

In [31]:
movies.query('release_date >= @cpi.date.min() & release_date <= @cpi.date.max()')

,movie_id,adult,original_title,runtime,popularity,revenue,vote_average,release_date,DimCastID,DimCrewID,DimDateID,DimGenreID,DimKeywordsID,DimProdCompID
1537,131232,False,Due Amici,86.0,0.003949,0.0,0.0,2002-03-20,40906.0,26029.0,708,19785.0,-1.0,-1.0
1878,79782,False,Wenecja,110.0,0.147130,0.0,7.5,2010-05-25,32050.0,24350.0,869,19846.0,-1.0,-1.0
2234,141210,False,The Sleepover,6.0,0.135596,0.0,8.0,2013-10-12,36222.0,41724.0,1082,18302.0,-1.0,-1.0
3461,216794,False,Men of Means,80.0,0.022078,0.0,0.0,1999-05-25,36179.0,33685.0,1655,18422.0,19810.0,16206.0
4320,143750,False,The Farmer's Wife,18.0,0.211754,0.0,10.0,2012-06-20,39124.0,17040.0,2114,30377.0,25106.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82360,390959,False,Shadow of the Blair Witch,45.0,0.076061,0.0,7.0,2000-10-22,7132.0,2723.0,45425,19432.0,25131.0,-1.0
82361,289923,False,The Burkittsville 7,30.0,0.386450,0.0,7.0,2000-10-03,28642.0,35547.0,45426,29197.0,10473.0,15671.0
82369,111109,False,Siglo ng Pagluluwal,360.0,0.178241,0.0,9.0,2011-11-17,19324.0,21491.0,45430,29180.0,17700.0,17981.0
82370,67758,False,Betrayal,90.0,0.903007,0.0,3.8,2003-08-01,13072.0,25035.0,45431,10921.0,-1.0,17980.0


We can normalize revenue of 24721 movies and almost 20000 of them have 0 revenue 

Not worth it<br>
But if you interested you can do the step [here](https://dof.ca.gov/wp-content/uploads/Forecasting/Economics/Documents/How-To-Use-CPI-Data.pdf)

### convert Dim ID to Int

In [32]:
movies[['DimCastID','DimCrewID','DimGenreID','DimKeywordsID','DimProdCompID']] = \
movies[['DimCastID','DimCrewID','DimGenreID','DimKeywordsID','DimProdCompID']].astype('int64')

#### save the data

In [33]:
movies.to_csv('data/out/Fact.csv',index=False)